In [1]:
import numpy as np
import random
from IPython.display import clear_output
from collections import deque
import progressbar
import gym
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

In [11]:
env =gym.make("Taxi-v3").env

In [13]:
print('Number of state: {}'.format(env.observation_space.n))
print('Number of state: {}'.format(env.action_space.n))

Number of state: 500
Number of state: 6


In [26]:
class Agent:
    def __init__(self, env, optimizer):
        
        self._state_size = env.observation_space.n
        self._action_size = env.action_space.n
        self._optimizer = optimizer

        self.expirience_reply = deque(maxlen=2000)
        
        self.gamma = 0.6
        self.epsilon = 0.1

        self.q_network = self.build_compile_model()
        self.target_network = self.build_compile_model()
        self.alighn_target_model


    def store(self, state, action, reward, next_state, terminated):
        self.expirience_reply.append(state, action, reward, next_state, terminated)
    
    def build_compile_model(self):
        model = Sequential()
        model.add(Embedding(self._state_size, 10, input_length=1))
        model.add(Reshape((10,)))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))

        model.compile(loss='mse', optimizer=self._optimizer)
        return model    
    
    def alighn_target_model(self):
        self.target_network.set_weights(self.q_network.get_weights())
    
    def act(self, state):

        if np.random.rand () <= self.epsilon:
            return env.action_space.sample()
        q_vales = self.q_network.predict(state)
        return np.argmax(q_vales[0])            
    
    def retrain(self, batch_size):
        miniBatch = random.sample(self.expirience_reply, batch_size)

        for state, action, reward, next_state, terminated in miniBatch:
            
            target = self.q_network.predict(state)

            if terminated:
                target[0][action] = reward
            else:
                t = self.target_network.predict(next_state)
                target[0][action] = reward + self.gamma * np.amax(t)
            
            self.q_network.fit(state, target, epochs=1, verbose=1)
 

In [27]:
optimizer = Adam(learning_rate=0.01)
agent= Agent(env, optimizer) 
batch_size = 32
number_of_episode =100
timeSteps = 1000
agent.q_network.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 10)             5000      
                                                                 
 reshape_2 (Reshape)         (None, 10)                0         
                                                                 
 dense_6 (Dense)             (None, 50)                550       
                                                                 
 dense_7 (Dense)             (None, 50)                2550      
                                                                 
 dense_8 (Dense)             (None, 6)                 306       
                                                                 
Total params: 8,406
Trainable params: 8,406
Non-trainable params: 0
_________________________________________________________________


In [30]:
for e in range(number_of_episode):

    state = env.reset()[0]
    state = np.reshape(state, [1,1])

    reward = 0
    terminated = False

    bar = progressbar.ProgressBar(maxval=timeSteps/10, widgets= [progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()

    for timeSteps in range(timeSteps):
        action = agent.act(state)

        next_state, reward, terminated = env.step(action)
        next_state = np.reshape(next_state, [1,1])
        agent.store(state, action, reward, next_state, terminated)

        state = next_state

        if terminated:
            agent.alighn_target_model()
            break
        if len(agent.expirience_reply) > batch_size:
            agent.retrain(batch_size)

        if timeSteps%10 ==0:
            bar.update(timeSteps/10+1)
        
        bar.finish()
        if (e+1) % 10 ==0:
            print("************************")
            print("Episode: {}".format(e+1) )
            env.render()
            print("************************")
